In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import threading
import gc
import copy

sys.path.append("/end/home/rh2310/morpho_repo/turing_codebase")
from turing.utils import *
import turing.pinns as tu
from turing.loss_functions import *

GPU = False
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"

2022-07-25 15:08:30.879075: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-07-25 15:08:43.740901: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-07-25 15:08:43.766368: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-07-25 15:08:43.766426: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-07-25 15:08:43.775152: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-07-25 15:08:43.781764: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10

# Load data

In [2]:
data_path = os.path.abspath("turing.npy")
with open(data_path, 'rb') as f:
    data = np.load(f)
    
data_path = os.path.abspath("turing_t.npy")
with open(data_path, 'rb') as f:
    t_star = np.load(f) 
    
T = t_star.shape[0]    
    
L = 50
x_size = data.shape[1]
y_size = data.shape[2]
N = x_size*y_size

model_params = {'training_data_size': T*N,#T*32,
                'pde_data_size': (T*N)//(32),
                'boundary_data_size':((x_size + y_size)*T)//(8)}

dataset = create_dataset(data, t_star, N, T, L, **model_params)
lb = dataset['lb']
ub = dataset['ub']
obs_X = dataset['obs_input']
obs_Y = dataset['obs_output']
pde_X = dataset['pde']


In [3]:
data_path = os.path.abspath("turing_model_2.npy")
with open(data_path, 'rb') as f:
    data2 = np.load(f)
    
data_path = os.path.abspath("turing_t_model_2.npy")
with open(data_path, 'rb') as f:
    t_star2 = np.load(f) 
    
T2 = t_star2.shape[0]    
    
L2 = 50
x_size2 = data2.shape[1]
y_size2 = data2.shape[2]
N2 = x_size2*y_size2

model_params2 = {'training_data_size': T2*N2,#T*32,
                'pde_data_size': (T2*N2)//(32),
                'boundary_data_size':((x_size2 + y_size2)*T2)//(8)}

dataset2 = create_dataset(data2, t_star2, N2, T2, L2, **model_params2)
lb2 = dataset['lb']
ub2 = dataset['ub']
obs_X2 = dataset2['obs_input']
obs_Y2 = dataset2['obs_output']
pde_X2 = dataset2['pde']

# Test 

In [10]:
layers = [3, 64, 64, 64, 64, 2]

pinn = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss = ASDM(dtype=tf.float64, D_a = 0.005, D_s = 0.2)

In [11]:
model = tu.TINN(pinn, pde_loss, alpha = 0.25)

In [12]:
results = model.train(epochs = 500,
                      batch_size = 512,
                      X = obs_X,
                      Y = obs_Y,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 4.0110
total loss: 25540.56468, total regularisd loss: 25540.56468
obs u loss: 583.38952, obs v loss: 13697.76128
pde u loss: 451.09257, pde v loss: 10808.32131
lambda obs u: 1.00000, lambda obs v: 1.00000
lambda pde u: 1.00000, lambda pde v: 1.00000
sigma_a:0:10.02714023, sigma_s:0:8.43172385, mu_a:0:9.79288470, rho_a:0:10.38194313, rho_s:0:11.83654228, kappa_a:0:8.04983544
Time taken: 48.70s

Start of epoch 1
Training observations acc over epoch: 4.1434
total loss: 11816.42264, total regularisd loss: 11816.42264
obs u loss: 592.33718, obs v loss: 7893.39371
pde u loss: 0.69590, pde v loss: 3329.99585
lambda obs u: 0.33306, lambda obs v: 2.75501
lambda pde u: 0.25000, lambda pde v: 0.66193
sigma_a:0:10.07004369, sigma_s:0:7.61199094, mu_a:0:9.69885849, rho_a:0:10.54549089, rho_s:0:12.64843414, kappa_a:0:7.09827691
Time taken: 32.78s

Start of epoch 2
Training observations acc over epoch: 0.9685
total loss: 13277.59580, total regu

Training observations acc over epoch: 0.1325
total loss: 24675.35869, total regularisd loss: 977.09384
obs u loss: 247.85191, obs v loss: 23.48482
pde u loss: 54.84619, pde v loss: 12.77024
lambda obs u: 3.94321, lambda obs v: 0.05153
lambda pde u: 0.00439, lambda pde v: 0.00088
sigma_a:0:8.36270425, sigma_s:0:4.27665489, mu_a:0:10.93609371, rho_a:0:9.22655265, rho_s:0:15.39068991, kappa_a:0:3.57435712
Time taken: 24.92s

Start of epoch 20
Training observations acc over epoch: 0.1326
total loss: 28819.25539, total regularisd loss: 978.60864
obs u loss: 247.80720, obs v loss: 23.67610
pde u loss: 50.34671, pde v loss: 14.75007
lambda obs u: 3.94620, lambda obs v: 0.04862
lambda pde u: 0.00413, lambda pde v: 0.00105
sigma_a:0:8.33229327, sigma_s:0:4.28410853, mu_a:0:10.91246189, rho_a:0:9.25006400, rho_s:0:15.37945116, kappa_a:0:3.55830894
Time taken: 24.98s

Start of epoch 21
Training observations acc over epoch: 0.1325
total loss: 25656.39001, total regularisd loss: 979.38681
obs u los

Training observations acc over epoch: 0.1324
total loss: 25447.83835, total regularisd loss: 977.57811
obs u loss: 247.77741, obs v loss: 23.38252
pde u loss: 43.12052, pde v loss: 13.69401
lambda obs u: 3.93797, lambda obs v: 0.05714
lambda pde u: 0.00395, lambda pde v: 0.00094
sigma_a:0:7.58793032, sigma_s:0:4.51174167, mu_a:0:10.55331915, rho_a:0:9.53890602, rho_s:0:14.35755646, kappa_a:0:3.04791491
Time taken: 24.90s

Start of epoch 39
Training observations acc over epoch: 0.1324
total loss: 26151.37664, total regularisd loss: 977.22928
obs u loss: 247.77072, obs v loss: 23.41013
pde u loss: 41.62934, pde v loss: 14.21471
lambda obs u: 3.93549, lambda obs v: 0.05946
lambda pde u: 0.00400, lambda pde v: 0.00105
sigma_a:0:7.52816816, sigma_s:0:4.51336087, mu_a:0:10.54475719, rho_a:0:9.54455685, rho_s:0:14.29971778, kappa_a:0:3.02579005
Time taken: 24.79s

Start of epoch 40
Training observations acc over epoch: 0.1324
total loss: 24956.81836, total regularisd loss: 976.92826
obs u los

Training observations acc over epoch: 0.1323
total loss: 25402.67214, total regularisd loss: 975.54613
obs u loss: 247.74017, obs v loss: 23.18069
pde u loss: 35.86446, pde v loss: 13.90504
lambda obs u: 3.92966, lambda obs v: 0.06577
lambda pde u: 0.00359, lambda pde v: 0.00098
sigma_a:0:6.62133533, sigma_s:0:4.74407639, mu_a:0:10.22276767, rho_a:0:9.77568235, rho_s:0:13.05733625, kappa_a:0:2.49188647
Time taken: 25.07s

Start of epoch 58
Training observations acc over epoch: 0.1323
total loss: 25052.48681, total regularisd loss: 975.17073
obs u loss: 247.73216, obs v loss: 23.18650
pde u loss: 35.50087, pde v loss: 14.43203
lambda obs u: 3.92873, lambda obs v: 0.06655
lambda pde u: 0.00369, lambda pde v: 0.00102
sigma_a:0:6.56345734, sigma_s:0:4.76291395, mu_a:0:10.20405374, rho_a:0:9.78734369, rho_s:0:12.97566292, kappa_a:0:2.46083428
Time taken: 25.34s

Start of epoch 59
Training observations acc over epoch: 0.1323
total loss: 23878.21453, total regularisd loss: 975.03533
obs u los

Training observations acc over epoch: 0.1322
total loss: 24201.74505, total regularisd loss: 973.53311
obs u loss: 247.71885, obs v loss: 22.98820
pde u loss: 29.72738, pde v loss: 14.49263
lambda obs u: 3.92148, lambda obs v: 0.07453
lambda pde u: 0.00300, lambda pde v: 0.00099
sigma_a:0:5.60000047, sigma_s:0:5.02807642, mu_a:0:9.87727249, rho_a:0:9.94916526, rho_s:0:11.64387537, kappa_a:0:1.94501307
Time taken: 24.96s

Start of epoch 77
Training observations acc over epoch: 0.1322
total loss: 24763.56869, total regularisd loss: 973.24033
obs u loss: 247.71910, obs v loss: 22.97236
pde u loss: 29.39977, pde v loss: 14.40444
lambda obs u: 3.91814, lambda obs v: 0.07784
lambda pde u: 0.00303, lambda pde v: 0.00099
sigma_a:0:5.53205759, sigma_s:0:5.03771576, mu_a:0:9.86639717, rho_a:0:9.94801966, rho_s:0:11.57219932, kappa_a:0:1.92284500
Time taken: 24.91s

Start of epoch 78
Training observations acc over epoch: 0.1322
total loss: 24882.84947, total regularisd loss: 972.59288
obs u loss:

...

Training observations acc over epoch: 0.0021
total loss: 8.09959, total regularisd loss: 9.29401
obs u loss: 2.79869, obs v loss: 1.49487
pde u loss: 3.79881, pde v loss: 0.39463
lambda obs u: 1.39637, lambda obs v: 1.09495
lambda pde u: 0.86113, lambda pde v: 0.64755
sigma_a:0:0.85651891, sigma_s:0:0.01882502, mu_a:0:4.49773412, rho_a:0:3.90176546, rho_s:0:0.02072651, kappa_a:0:0.30793386
Time taken: 25.84s

Start of epoch 419
Training observations acc over epoch: 0.0020
total loss: 8.20686, total regularisd loss: 8.77071
obs u loss: 2.62079, obs v loss: 1.46215
pde u loss: 3.78360, pde v loss: 0.38911
lambda obs u: 1.34286, lambda obs v: 1.10300
lambda pde u: 0.89013, lambda pde v: 0.66401
sigma_a:0:0.84275832, sigma_s:0:0.01954047, mu_a:0:4.44939684, rho_a:0:3.86687156, rho_s:0:0.02047024, kappa_a:0:0.30789488
Time taken: 25.50s

Start of epoch 420
Training observations acc over epoch: 0.0020
total loss: 7.87894, total regularisd loss: 8.56023
obs u loss: 2.60808, obs v loss: 1.4521

Training observations acc over epoch: 0.0014
total loss: 5.68205, total regularisd loss: 5.75262
obs u loss: 1.73465, obs v loss: 1.18158
pde u loss: 2.39084, pde v loss: 0.32538
lambda obs u: 1.04225, lambda obs v: 1.24249
lambda pde u: 0.95471, lambda pde v: 0.76055
sigma_a:0:0.66047178, sigma_s:0:0.01904837, mu_a:0:3.51208497, rho_a:0:3.03861728, rho_s:0:0.02057769, kappa_a:0:0.30174175
Time taken: 25.67s

Start of epoch 438
Training observations acc over epoch: 0.0014
total loss: 5.37979, total regularisd loss: 5.65929
obs u loss: 1.77742, obs v loss: 1.17904
pde u loss: 2.19390, pde v loss: 0.32515
lambda obs u: 1.12741, lambda obs v: 1.23981
lambda pde u: 0.86356, lambda pde v: 0.76921
sigma_a:0:0.65511563, sigma_s:0:0.02020321, mu_a:0:3.45701133, rho_a:0:2.98310303, rho_s:0:0.02002341, kappa_a:0:0.30121278
Time taken: 25.36s

Start of epoch 439
Training observations acc over epoch: 0.0014
total loss: 5.46888, total regularisd loss: 5.52394
obs u loss: 1.66424, obs v loss: 1.1494

Training observations acc over epoch: 0.0010
total loss: 3.45890, total regularisd loss: 3.67495
obs u loss: 1.18959, obs v loss: 0.84471
pde u loss: 1.24471, pde v loss: 0.24875
lambda obs u: 0.92237, lambda obs v: 1.21157
lambda pde u: 1.07030, lambda pde v: 0.79575
sigma_a:0:0.41118411, sigma_s:0:0.01950168, mu_a:0:2.17720315, rho_a:0:1.84972030, rho_s:0:0.01956139, kappa_a:0:0.28529803
Time taken: 25.60s

Start of epoch 457
Training observations acc over epoch: 0.0009
total loss: 3.33992, total regularisd loss: 3.52867
obs u loss: 1.11294, obs v loss: 0.79622
pde u loss: 1.25916, pde v loss: 0.23848
lambda obs u: 0.84071, lambda obs v: 1.17142
lambda pde u: 1.19949, lambda pde v: 0.78839
sigma_a:0:0.39079614, sigma_s:0:0.01944574, mu_a:0:2.08646089, rho_a:0:1.76843598, rho_s:0:0.01965643, kappa_a:0:0.28397208
Time taken: 25.54s

Start of epoch 458
Training observations acc over epoch: 0.0009
total loss: 3.25047, total regularisd loss: 3.40188
obs u loss: 1.13256, obs v loss: 0.7850

Training observations acc over epoch: 0.0003
total loss: 0.74971, total regularisd loss: 1.01846
obs u loss: 0.47581, obs v loss: 0.18024
pde u loss: 0.00702, pde v loss: 0.09741
lambda obs u: 1.48163, lambda obs v: 0.64938
lambda pde u: 0.03528, lambda pde v: 1.83371
sigma_a:0:0.00797549, sigma_s:0:0.01803798, mu_a:0:0.06309769, rho_a:0:0.06083584, rho_s:0:0.02018163, kappa_a:0:0.33784677
Time taken: 25.67s

Start of epoch 476
Training observations acc over epoch: 0.0003
total loss: 0.77869, total regularisd loss: 0.97688
obs u loss: 0.46472, obs v loss: 0.17499
pde u loss: 0.00507, pde v loss: 0.09517
lambda obs u: 1.51512, lambda obs v: 0.66518
lambda pde u: 0.02159, lambda pde v: 1.79812
sigma_a:0:0.00533539, sigma_s:0:0.01851507, mu_a:0:0.04868485, rho_a:0:0.04887603, rho_s:0:0.02125392, kappa_a:0:0.36082856
Time taken: 25.35s

Start of epoch 477
Training observations acc over epoch: 0.0003
total loss: 0.78251, total regularisd loss: 0.99847
obs u loss: 0.47200, obs v loss: 0.1714

Training observations acc over epoch: 0.0002
total loss: 1.58636, total regularisd loss: 0.76331
obs u loss: 0.35860, obs v loss: 0.13251
pde u loss: 0.00073, pde v loss: 0.08130
lambda obs u: 1.45200, lambda obs v: 0.54912
lambda pde u: 0.00069, lambda pde v: 1.99818
sigma_a:0:0.00000000, sigma_s:0:0.01772542, mu_a:0:0.00982541, rho_a:0:0.01054125, rho_s:0:0.01913531, kappa_a:0:0.31098076
Time taken: 25.70s

Start of epoch 495
Training observations acc over epoch: 0.0002
total loss: 1.56413, total regularisd loss: 0.74592
obs u loss: 0.35319, obs v loss: 0.13065
pde u loss: 0.00072, pde v loss: 0.08075
lambda obs u: 1.47491, lambda obs v: 0.56305
lambda pde u: 0.00073, lambda pde v: 1.96131
sigma_a:0:0.00000000, sigma_s:0:0.01735825, mu_a:0:0.00976734, rho_a:0:0.01060954, rho_s:0:0.01994805, kappa_a:0:0.30856900
Time taken: 25.72s

Start of epoch 496
Training observations acc over epoch: 0.0002
total loss: 1.52614, total regularisd loss: 0.78086
obs u loss: 0.37222, obs v loss: 0.1307

In [4]:
layers = [3, 64, 64, 64, 64, 2]
pinn2 = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss2 = ASDM(dtype=tf.float64, D_a = 1.0, D_s = 40.0)

In [5]:
model2 = tu.TINN(pinn2, pde_loss2)

In [6]:
results2 = model2.train(epochs = 500,
                      batch_size = 512,
                      X = obs_X2,
                      Y = obs_Y2,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0


2022-07-25 15:09:15.544917: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-07-25 15:09:22.159523: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


Training observations acc over epoch: 9.42273
total loss: 39.05624, total regularisd loss (sum of batches): 39993.59077
obs u loss: 1.47133, obs v loss: 17.37413
pde u loss: 1.04630, pde v loss: 19.16448
lambda obs u: 1.00000, lambda obs v: 1.00000
lambda pde u: 1.00000, lambda pde v: 1.00000
sigma_a: 10.02170 sigma_s: 9.22966 mu_a: 9.84105 rho_a: 10.56301 
rho_s: 10.97523 kappa_a: 8.98399 
Time taken: 38.13s

Start of epoch 1
Training observations acc over epoch: 6.32113
total loss: 18.22915, total regularisd loss (sum of batches): 18666.65150
obs u loss: 1.44113, obs v loss: 11.20113
pde u loss: 0.00775, pde v loss: 5.57914
lambda obs u: 0.67661, lambda obs v: 2.03779
lambda pde u: 0.50001, lambda pde v: 0.78559
sigma_a: 10.05966 sigma_s: 8.56213 mu_a: 9.70183 rho_a: 10.87078 
rho_s: 11.64897 kappa_a: 8.22789 
Time taken: 25.53s

Start of epoch 2
Training observations acc over epoch: 2.58177
total loss: 12.97510, total regularisd loss (sum of batches): 15161.54623
obs u loss: 1.35675

Training observations acc over epoch: 0.12501
total loss: 0.69725, total regularisd loss (sum of batches): 605.52906
obs u loss: 0.07759, obs v loss: 0.17243
pde u loss: 0.25091, pde v loss: 0.19632
lambda obs u: 1.27055, lambda obs v: 1.60969
lambda pde u: 0.60421, lambda pde v: 0.51555
sigma_a: 0.00000 sigma_s: 0.72519 mu_a: 1.95214 rho_a: 17.92876 
rho_s: 6.57283 kappa_a: 0.35706 
Time taken: 21.88s

Start of epoch 21
Training observations acc over epoch: 0.12096
total loss: 0.59967, total regularisd loss (sum of batches): 573.10545
obs u loss: 0.09385, obs v loss: 0.14807
pde u loss: 0.19911, pde v loss: 0.15864
lambda obs u: 1.21559, lambda obs v: 1.56259
lambda pde u: 0.67333, lambda pde v: 0.54849
sigma_a: 0.00000 sigma_s: 0.61544 mu_a: 1.89303 rho_a: 17.40218 
rho_s: 5.81516 kappa_a: 0.45213 
Time taken: 20.24s

Start of epoch 22
Training observations acc over epoch: 0.11098
total loss: 0.52928, total regularisd loss (sum of batches): 514.46682
obs u loss: 0.10080, obs v loss: 

Training observations acc over epoch: 0.01977
total loss: 0.10061, total regularisd loss (sum of batches): 85.14892
obs u loss: 0.01768, obs v loss: 0.02187
pde u loss: 0.05171, pde v loss: 0.00936
lambda obs u: 1.71340, lambda obs v: 1.01880
lambda pde u: 0.43004, lambda pde v: 0.83776
sigma_a: 1.02011 sigma_s: 0.01633 mu_a: 5.46877 rho_a: 9.22542 
rho_s: 0.06593 kappa_a: 0.12322 
Time taken: 20.21s

Start of epoch 41
Training observations acc over epoch: 0.01874
total loss: 0.09707, total regularisd loss (sum of batches): 81.24271
obs u loss: 0.01713, obs v loss: 0.02035
pde u loss: 0.05069, pde v loss: 0.00890
lambda obs u: 1.76667, lambda obs v: 0.98148
lambda pde u: 0.43303, lambda pde v: 0.81882
sigma_a: 1.06338 sigma_s: 0.01632 mu_a: 5.53471 rho_a: 9.03298 
rho_s: 0.07183 kappa_a: 0.12035 
Time taken: 20.44s

Start of epoch 42
Training observations acc over epoch: 0.01794
total loss: 0.09273, total regularisd loss (sum of batches): 78.12376
obs u loss: 0.01677, obs v loss: 0.019

...

Training observations acc over epoch: 0.00454
total loss: 0.02064, total regularisd loss (sum of batches): 18.57746
obs u loss: 0.00551, obs v loss: 0.00357
pde u loss: 0.00731, pde v loss: 0.00425
lambda obs u: 1.16304, lambda obs v: 1.98812
lambda pde u: 0.33766, lambda pde v: 0.51118
sigma_a: 0.25064 sigma_s: 0.26370 mu_a: 1.60445 rho_a: 1.63185 
rho_s: 0.34946 kappa_a: 0.03033 
Time taken: 20.34s

Start of epoch 401
Training observations acc over epoch: 0.00451
total loss: 0.02053, total regularisd loss (sum of batches): 18.47571
obs u loss: 0.00548, obs v loss: 0.00355
pde u loss: 0.00729, pde v loss: 0.00421
lambda obs u: 1.15957, lambda obs v: 2.00092
lambda pde u: 0.33499, lambda pde v: 0.50452
sigma_a: 0.24922 sigma_s: 0.26549 mu_a: 1.60523 rho_a: 1.63039 
rho_s: 0.34953 kappa_a: 0.02973 
Time taken: 19.37s

Start of epoch 402
Training observations acc over epoch: 0.00452
total loss: 0.02070, total regularisd loss (sum of batches): 18.52147
obs u loss: 0.00549, obs v loss: 0.0

Training observations acc over epoch: 0.00438
total loss: 0.02003, total regularisd loss (sum of batches): 18.01662
obs u loss: 0.00538, obs v loss: 0.00338
pde u loss: 0.00717, pde v loss: 0.00410
lambda obs u: 1.20702, lambda obs v: 1.94997
lambda pde u: 0.33363, lambda pde v: 0.50938
sigma_a: 0.24478 sigma_s: 0.27679 mu_a: 1.59135 rho_a: 1.61984 
rho_s: 0.37325 kappa_a: 0.02917 
Time taken: 21.37s

Start of epoch 421
Training observations acc over epoch: 0.00435
total loss: 0.01997, total regularisd loss (sum of batches): 17.91358
obs u loss: 0.00535, obs v loss: 0.00336
pde u loss: 0.00712, pde v loss: 0.00415
lambda obs u: 1.19808, lambda obs v: 1.95569
lambda pde u: 0.34008, lambda pde v: 0.50615
sigma_a: 0.24450 sigma_s: 0.27682 mu_a: 1.59170 rho_a: 1.61949 
rho_s: 0.37473 kappa_a: 0.02930 
Time taken: 20.90s

Start of epoch 422
Training observations acc over epoch: 0.00433
total loss: 0.01992, total regularisd loss (sum of batches): 17.85196
obs u loss: 0.00532, obs v loss: 0.0

Training observations acc over epoch: 0.00427
total loss: 0.01964, total regularisd loss (sum of batches): 17.62507
obs u loss: 0.00530, obs v loss: 0.00324
pde u loss: 0.00697, pde v loss: 0.00413
lambda obs u: 1.23974, lambda obs v: 1.90828
lambda pde u: 0.34120, lambda pde v: 0.51079
sigma_a: 0.24354 sigma_s: 0.28640 mu_a: 1.58580 rho_a: 1.61369 
rho_s: 0.39220 kappa_a: 0.02850 
Time taken: 21.11s

Start of epoch 441
Training observations acc over epoch: 0.00430
total loss: 0.01993, total regularisd loss (sum of batches): 17.84482
obs u loss: 0.00537, obs v loss: 0.00324
pde u loss: 0.00705, pde v loss: 0.00427
lambda obs u: 1.23556, lambda obs v: 1.90068
lambda pde u: 0.35003, lambda pde v: 0.51373
sigma_a: 0.24194 sigma_s: 0.28613 mu_a: 1.58539 rho_a: 1.61388 
rho_s: 0.39292 kappa_a: 0.02853 
Time taken: 21.69s

Start of epoch 442
Training observations acc over epoch: 0.00426
total loss: 0.01965, total regularisd loss (sum of batches): 17.65100
obs u loss: 0.00527, obs v loss: 0.0

Training observations acc over epoch: 0.00418
total loss: 0.01951, total regularisd loss (sum of batches): 17.42184
obs u loss: 0.00523, obs v loss: 0.00313
pde u loss: 0.00690, pde v loss: 0.00425
lambda obs u: 1.24895, lambda obs v: 1.88358
lambda pde u: 0.34949, lambda pde v: 0.51797
sigma_a: 0.23877 sigma_s: 0.29874 mu_a: 1.57466 rho_a: 1.59985 
rho_s: 0.40642 kappa_a: 0.02827 
Time taken: 21.38s

Start of epoch 461
Training observations acc over epoch: 0.00422
total loss: 0.01975, total regularisd loss (sum of batches): 17.62480
obs u loss: 0.00529, obs v loss: 0.00315
pde u loss: 0.00703, pde v loss: 0.00428
lambda obs u: 1.27632, lambda obs v: 1.86697
lambda pde u: 0.34516, lambda pde v: 0.51155
sigma_a: 0.23997 sigma_s: 0.29677 mu_a: 1.57289 rho_a: 1.59933 
rho_s: 0.40363 kappa_a: 0.02811 
Time taken: 21.69s

Start of epoch 462
Training observations acc over epoch: 0.00417
total loss: 0.01908, total regularisd loss (sum of batches): 17.24756
obs u loss: 0.00521, obs v loss: 0.0

Training observations acc over epoch: 0.00407
total loss: 0.01901, total regularisd loss (sum of batches): 17.02821
obs u loss: 0.00513, obs v loss: 0.00302
pde u loss: 0.00680, pde v loss: 0.00406
lambda obs u: 1.27915, lambda obs v: 1.86184
lambda pde u: 0.34361, lambda pde v: 0.51540
sigma_a: 0.23691 sigma_s: 0.30886 mu_a: 1.56157 rho_a: 1.58531 
rho_s: 0.42237 kappa_a: 0.02717 
Time taken: 21.73s

Start of epoch 481
Training observations acc over epoch: 0.00408
total loss: 0.01903, total regularisd loss (sum of batches): 17.02667
obs u loss: 0.00515, obs v loss: 0.00301
pde u loss: 0.00680, pde v loss: 0.00408
lambda obs u: 1.28249, lambda obs v: 1.84064
lambda pde u: 0.35100, lambda pde v: 0.52588
sigma_a: 0.23665 sigma_s: 0.31245 mu_a: 1.56004 rho_a: 1.58506 
rho_s: 0.42366 kappa_a: 0.02733 
Time taken: 21.69s

Start of epoch 482
Training observations acc over epoch: 0.00408
total loss: 0.01900, total regularisd loss (sum of batches): 17.05805
obs u loss: 0.00514, obs v loss: 0.0

In [7]:
for para in pde_loss2.trainables():
    print(f"{para.name}: {para.numpy()[0]}")

sigma_a:0: 0.23388787403732503
sigma_s:0: 0.32419621991885006
mu_a:0: 1.5515899607761958
rho_a:0: 1.5740713145714178
rho_s:0: 0.43720996633334314
kappa_a:0: 0.026605874590398395


In [8]:
results2_2 = model2.train(epochs = 100,
                      batch_size = 512,
                      X = obs_X2,
                      Y = obs_Y2,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 0.00399
total loss: 0.01864, total regularisd loss (sum of batches): 16.70885
obs u loss: 0.00508, obs v loss: 0.00291
pde u loss: 0.00661, pde v loss: 0.00405
lambda obs u: 1.29372, lambda obs v: 1.83861
lambda pde u: 0.34927, lambda pde v: 0.51840
sigma_a: 0.23279 sigma_s: 0.32316 mu_a: 1.55127 rho_a: 1.57336 
rho_s: 0.43756 kappa_a: 0.02649 
Time taken: 21.79s

Start of epoch 1
Training observations acc over epoch: 0.00398
total loss: 0.01871, total regularisd loss (sum of batches): 16.72351
obs u loss: 0.00506, obs v loss: 0.00290
pde u loss: 0.00660, pde v loss: 0.00416
lambda obs u: 1.28525, lambda obs v: 1.83932
lambda pde u: 0.35554, lambda pde v: 0.51990
sigma_a: 0.23431 sigma_s: 0.32314 mu_a: 1.55071 rho_a: 1.57278 
rho_s: 0.43400 kappa_a: 0.02676 
Time taken: 20.97s

Start of epoch 2
Training observations acc over epoch: 0.00399
total loss: 0.01863, total regularisd loss (sum of batches): 16.71091
obs u loss: 0.00508, o

Training observations acc over epoch: 0.00388
total loss: 0.01824, total regularisd loss (sum of batches): 16.41352
obs u loss: 0.00491, obs v loss: 0.00285
pde u loss: 0.00649, pde v loss: 0.00399
lambda obs u: 1.32230, lambda obs v: 1.79057
lambda pde u: 0.35571, lambda pde v: 0.53142
sigma_a: 0.23057 sigma_s: 0.33239 mu_a: 1.53900 rho_a: 1.55836 
rho_s: 0.44360 kappa_a: 0.02605 
Time taken: 21.27s

Start of epoch 21
Training observations acc over epoch: 0.00392
total loss: 0.01848, total regularisd loss (sum of batches): 16.59597
obs u loss: 0.00500, obs v loss: 0.00285
pde u loss: 0.00656, pde v loss: 0.00407
lambda obs u: 1.32295, lambda obs v: 1.78081
lambda pde u: 0.35802, lambda pde v: 0.53822
sigma_a: 0.23178 sigma_s: 0.33238 mu_a: 1.53846 rho_a: 1.55780 
rho_s: 0.44539 kappa_a: 0.02584 
Time taken: 21.60s

Start of epoch 22
Training observations acc over epoch: 0.00389
total loss: 0.01822, total regularisd loss (sum of batches): 16.44356
obs u loss: 0.00494, obs v loss: 0.002

Training observations acc over epoch: 0.00383
total loss: 0.01796, total regularisd loss (sum of batches): 16.20114
obs u loss: 0.00491, obs v loss: 0.00274
pde u loss: 0.00634, pde v loss: 0.00396
lambda obs u: 1.33533, lambda obs v: 1.77440
lambda pde u: 0.35832, lambda pde v: 0.53194
sigma_a: 0.22923 sigma_s: 0.34635 mu_a: 1.53064 rho_a: 1.54707 
rho_s: 0.45505 kappa_a: 0.02596 
Time taken: 21.36s

Start of epoch 41
Training observations acc over epoch: 0.00383
total loss: 0.01806, total regularisd loss (sum of batches): 16.25497
obs u loss: 0.00492, obs v loss: 0.00275
pde u loss: 0.00633, pde v loss: 0.00407
lambda obs u: 1.32538, lambda obs v: 1.78816
lambda pde u: 0.35505, lambda pde v: 0.53141
sigma_a: 0.22863 sigma_s: 0.34576 mu_a: 1.53064 rho_a: 1.54595 
rho_s: 0.45409 kappa_a: 0.02558 
Time taken: 21.87s

Start of epoch 42
Training observations acc over epoch: 0.00379
total loss: 0.01777, total regularisd loss (sum of batches): 15.99882
obs u loss: 0.00485, obs v loss: 0.002

Training observations acc over epoch: 0.00377
total loss: 0.01835, total regularisd loss (sum of batches): 16.25562
obs u loss: 0.00486, obs v loss: 0.00268
pde u loss: 0.00651, pde v loss: 0.00430
lambda obs u: 1.34821, lambda obs v: 1.73830
lambda pde u: 0.35397, lambda pde v: 0.55951
sigma_a: 0.22824 sigma_s: 0.35572 mu_a: 1.52040 rho_a: 1.53242 
rho_s: 0.46149 kappa_a: 0.02513 
Time taken: 28.12s

Start of epoch 61
Training observations acc over epoch: 0.00372
total loss: 0.01748, total regularisd loss (sum of batches): 15.81860
obs u loss: 0.00476, obs v loss: 0.00269
pde u loss: 0.00610, pde v loss: 0.00393
lambda obs u: 1.32970, lambda obs v: 1.77095
lambda pde u: 0.36766, lambda pde v: 0.53168
sigma_a: 0.22713 sigma_s: 0.35922 mu_a: 1.52143 rho_a: 1.53281 
rho_s: 0.46730 kappa_a: 0.02472 
Time taken: 26.61s

Start of epoch 62
Training observations acc over epoch: 0.00373
total loss: 0.01778, total regularisd loss (sum of batches): 15.91835
obs u loss: 0.00481, obs v loss: 0.002

Training observations acc over epoch: 0.00363
total loss: 0.01738, total regularisd loss (sum of batches): 15.54202
obs u loss: 0.00469, obs v loss: 0.00257
pde u loss: 0.00611, pde v loss: 0.00401
lambda obs u: 1.32278, lambda obs v: 1.74867
lambda pde u: 0.36272, lambda pde v: 0.56582
sigma_a: 0.22411 sigma_s: 0.37232 mu_a: 1.51292 rho_a: 1.52168 
rho_s: 0.47920 kappa_a: 0.02397 
Time taken: 28.04s

Start of epoch 81
Training observations acc over epoch: 0.00363
total loss: 0.01735, total regularisd loss (sum of batches): 15.53121
obs u loss: 0.00467, obs v loss: 0.00258
pde u loss: 0.00610, pde v loss: 0.00400
lambda obs u: 1.33572, lambda obs v: 1.73767
lambda pde u: 0.36159, lambda pde v: 0.56502
sigma_a: 0.22355 sigma_s: 0.37095 mu_a: 1.51252 rho_a: 1.52087 
rho_s: 0.47922 kappa_a: 0.02358 
Time taken: 28.27s

Start of epoch 82
Training observations acc over epoch: 0.00358
total loss: 0.01713, total regularisd loss (sum of batches): 15.35506
obs u loss: 0.00459, obs v loss: 0.002

In [9]:
for para in pde_loss2.trainables():
    print(f"{para.name}: {para.numpy()[0]}")

sigma_a:0: 0.22110723533454146
sigma_s:0: 0.3837675302024302
mu_a:0: 1.5067670530631199
rho_a:0: 1.509171987292396
rho_s:0: 0.49179947895617304
kappa_a:0: 0.023175854339708975


In [10]:
results2_3 = model2.train(epochs = 100,
                      batch_size = 512,
                      X = obs_X2,
                      Y = obs_Y2,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 0.00350
total loss: 0.01689, total regularisd loss (sum of batches): 15.05536
obs u loss: 0.00451, obs v loss: 0.00248
pde u loss: 0.00587, pde v loss: 0.00402
lambda obs u: 1.29720, lambda obs v: 1.76129
lambda pde u: 0.36984, lambda pde v: 0.57167
sigma_a: 0.22272 sigma_s: 0.38001 mu_a: 1.50522 rho_a: 1.50969 
rho_s: 0.48773 kappa_a: 0.02312 
Time taken: 28.08s

Start of epoch 1
Training observations acc over epoch: 0.00345
total loss: 0.01656, total regularisd loss (sum of batches): 14.81436
obs u loss: 0.00444, obs v loss: 0.00247
pde u loss: 0.00574, pde v loss: 0.00391
lambda obs u: 1.29043, lambda obs v: 1.78148
lambda pde u: 0.37090, lambda pde v: 0.55720
sigma_a: 0.22291 sigma_s: 0.38186 mu_a: 1.50546 rho_a: 1.50933 
rho_s: 0.49036 kappa_a: 0.02312 
Time taken: 28.34s

Start of epoch 2
Training observations acc over epoch: 0.00351
total loss: 0.01726, total regularisd loss (sum of batches): 15.22006
obs u loss: 0.00452, o

Training observations acc over epoch: 0.00332
total loss: 0.01613, total regularisd loss (sum of batches): 14.29672
obs u loss: 0.00427, obs v loss: 0.00238
pde u loss: 0.00561, pde v loss: 0.00387
lambda obs u: 1.29679, lambda obs v: 1.76869
lambda pde u: 0.36465, lambda pde v: 0.56987
sigma_a: 0.21947 sigma_s: 0.39737 mu_a: 1.50215 rho_a: 1.50154 
rho_s: 0.50305 kappa_a: 0.02247 
Time taken: 28.51s

Start of epoch 21
Training observations acc over epoch: 0.00334
total loss: 0.01632, total regularisd loss (sum of batches): 14.45887
obs u loss: 0.00428, obs v loss: 0.00239
pde u loss: 0.00563, pde v loss: 0.00402
lambda obs u: 1.28334, lambda obs v: 1.78568
lambda pde u: 0.35969, lambda pde v: 0.57128
sigma_a: 0.22044 sigma_s: 0.39740 mu_a: 1.50209 rho_a: 1.50176 
rho_s: 0.50368 kappa_a: 0.02237 
Time taken: 28.68s

Start of epoch 22
Training observations acc over epoch: 0.00331
total loss: 0.01617, total regularisd loss (sum of batches): 14.28845
obs u loss: 0.00426, obs v loss: 0.002

Training observations acc over epoch: 0.00324
total loss: 0.01578, total regularisd loss (sum of batches): 14.04835
obs u loss: 0.00412, obs v loss: 0.00236
pde u loss: 0.00546, pde v loss: 0.00383
lambda obs u: 1.28184, lambda obs v: 1.75380
lambda pde u: 0.36363, lambda pde v: 0.60073
sigma_a: 0.21668 sigma_s: 0.39877 mu_a: 1.49696 rho_a: 1.49621 
rho_s: 0.50194 kappa_a: 0.02222 
Time taken: 33.96s

Start of epoch 41
Training observations acc over epoch: 0.00325
total loss: 0.01576, total regularisd loss (sum of batches): 14.03337
obs u loss: 0.00414, obs v loss: 0.00235
pde u loss: 0.00547, pde v loss: 0.00380
lambda obs u: 1.28739, lambda obs v: 1.76954
lambda pde u: 0.35617, lambda pde v: 0.58690
sigma_a: 0.21776 sigma_s: 0.40272 mu_a: 1.49700 rho_a: 1.49488 
rho_s: 0.50736 kappa_a: 0.02204 
Time taken: 33.74s

Start of epoch 42
Training observations acc over epoch: 0.00322
total loss: 0.01559, total regularisd loss (sum of batches): 13.86093
obs u loss: 0.00413, obs v loss: 0.002

Training observations acc over epoch: 0.00317
total loss: 0.01546, total regularisd loss (sum of batches): 13.71943
obs u loss: 0.00407, obs v loss: 0.00226
pde u loss: 0.00530, pde v loss: 0.00383
lambda obs u: 1.30576, lambda obs v: 1.75821
lambda pde u: 0.34353, lambda pde v: 0.59250
sigma_a: 0.21452 sigma_s: 0.41494 mu_a: 1.48809 rho_a: 1.48330 
rho_s: 0.51628 kappa_a: 0.02151 
Time taken: 33.90s

Start of epoch 61
Training observations acc over epoch: 0.00314
total loss: 0.01537, total regularisd loss (sum of batches): 13.61809
obs u loss: 0.00404, obs v loss: 0.00224
pde u loss: 0.00525, pde v loss: 0.00383
lambda obs u: 1.30982, lambda obs v: 1.75464
lambda pde u: 0.34725, lambda pde v: 0.58829
sigma_a: 0.21620 sigma_s: 0.41506 mu_a: 1.48817 rho_a: 1.48273 
rho_s: 0.51750 kappa_a: 0.02133 
Time taken: 33.89s

Start of epoch 62
Training observations acc over epoch: 0.00319
total loss: 0.01543, total regularisd loss (sum of batches): 13.73436
obs u loss: 0.00413, obs v loss: 0.002

Training observations acc over epoch: 0.00313
total loss: 0.01530, total regularisd loss (sum of batches): 13.57715
obs u loss: 0.00405, obs v loss: 0.00221
pde u loss: 0.00519, pde v loss: 0.00385
lambda obs u: 1.31782, lambda obs v: 1.74019
lambda pde u: 0.34034, lambda pde v: 0.60165
sigma_a: 0.21325 sigma_s: 0.42063 mu_a: 1.48278 rho_a: 1.47700 
rho_s: 0.52086 kappa_a: 0.02103 
Time taken: 21.38s

Start of epoch 81
Training observations acc over epoch: 0.00312
total loss: 0.01503, total regularisd loss (sum of batches): 13.43722
obs u loss: 0.00405, obs v loss: 0.00220
pde u loss: 0.00507, pde v loss: 0.00372
lambda obs u: 1.32908, lambda obs v: 1.73642
lambda pde u: 0.34165, lambda pde v: 0.59285
sigma_a: 0.21107 sigma_s: 0.42432 mu_a: 1.48399 rho_a: 1.47512 
rho_s: 0.52512 kappa_a: 0.02089 
Time taken: 20.99s

Start of epoch 82
Training observations acc over epoch: 0.00310
total loss: 0.01503, total regularisd loss (sum of batches): 13.40898
obs u loss: 0.00402, obs v loss: 0.002

In [24]:
pinn3 = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss3 = ASDM(dtype=tf.float64, D_a = 1.0, D_s = 4.0)

In [25]:
model3 = tu.TINN(pinn3, pde_loss3)

In [26]:
results3 = model3.train(epochs = 500,
                      batch_size = 512,
                      X = obs_X2,
                      Y = obs_Y2,
                      print_interval=1,
                      stop_threshold = 1e-5,
                      shuffle = True,
                      sample_losses = True,              
                      sample_regularisations = True,
                      sample_gradients = True)


Start of epoch 0
Training observations acc over epoch: 8.8021
total loss: 31032.76525, total regularisd loss: 31032.76525
obs u loss: 1530.55521, obs v loss: 16496.17170
pde u loss: 466.76271, pde v loss: 12539.27564
lambda obs u: 1.00000, lambda obs v: 1.00000
lambda pde u: 1.00000, lambda pde v: 1.00000
sigma_a:0:10.05005491, sigma_s:0:8.38077383, mu_a:0:9.76782146, rho_a:0:10.34804045, rho_s:0:11.86291525, kappa_a:0:8.03814692
Time taken: 50.47s

Start of epoch 1
Training observations acc over epoch: 5.8171
total loss: 16211.61730, total regularisd loss: 16211.61730
obs u loss: 1552.07241, obs v loss: 10361.36601
pde u loss: 1.20419, pde v loss: 4296.97469
lambda obs u: 0.79394, lambda obs v: 1.96958
lambda pde u: 0.50000, lambda pde v: 0.73647
sigma_a:0:10.12610737, sigma_s:0:7.53224694, mu_a:0:9.61984442, rho_a:0:10.56274815, rho_s:0:12.69782776, kappa_a:0:7.05952550
Time taken: 33.74s

Start of epoch 2
Training observations acc over epoch: 2.3446
total loss: 12841.00384, total r

Training observations acc over epoch: 0.3920
total loss: 9812.62412, total regularisd loss: 2106.25308
obs u loss: 474.88512, obs v loss: 327.96130
pde u loss: 413.77241, pde v loss: 1015.93972
lambda obs u: 2.66899, lambda obs v: 0.17944
lambda pde u: 0.41779, lambda pde v: 0.73377
sigma_a:0:2.91596953, sigma_s:0:0.68026110, mu_a:0:3.64776424, rho_a:0:16.48610592, rho_s:0:10.15338291, kappa_a:0:1.00252107
Time taken: 26.19s

Start of epoch 20
Training observations acc over epoch: 0.3109
total loss: 4761.06769, total regularisd loss: 1443.55400
obs u loss: 253.71205, obs v loss: 382.91884
pde u loss: 764.25791, pde v loss: 515.67058
lambda obs u: 2.29517, lambda obs v: 0.55451
lambda pde u: 0.53254, lambda pde v: 0.61778
sigma_a:0:1.86176659, sigma_s:0:0.45876414, mu_a:0:2.42829506, rho_a:0:17.38376919, rho_s:0:9.10192179, kappa_a:0:0.58136984
Time taken: 26.33s

Start of epoch 21
Training observations acc over epoch: 0.2327
total loss: 1996.67015, total regularisd loss: 882.52119
obs 

Training observations acc over epoch: 0.0224
total loss: 75.20889, total regularisd loss: 107.54137
obs u loss: 31.97645, obs v loss: 13.97392
pde u loss: 33.04701, pde v loss: 3.74889
lambda obs u: 1.49638, lambda obs v: 0.65173
lambda pde u: 1.55890, lambda pde v: 0.29299
sigma_a:0:0.00000000, sigma_s:0:0.01209103, mu_a:0:0.00000000, rho_a:0:5.14099579, rho_s:0:0.90065769, kappa_a:0:9.71448766
Time taken: 25.91s

Start of epoch 39
Training observations acc over epoch: 0.0199
total loss: 67.65796, total regularisd loss: 99.68333
obs u loss: 29.37337, obs v loss: 11.46349
pde u loss: 30.35283, pde v loss: 3.21364
lambda obs u: 1.56176, lambda obs v: 0.66594
lambda pde u: 1.52549, lambda pde v: 0.24681
sigma_a:0:0.00000000, sigma_s:0:0.01727472, mu_a:0:0.00000000, rho_a:0:4.45574920, rho_s:0:1.01533427, kappa_a:0:9.86323978
Time taken: 26.33s

Start of epoch 40
Training observations acc over epoch: 0.0179
total loss: 61.42555, total regularisd loss: 93.61750
obs u loss: 27.39980, obs v 

Training observations acc over epoch: 0.0068
total loss: 25.06646, total regularisd loss: 36.72048
obs u loss: 11.73125, obs v loss: 2.14833
pde u loss: 7.87930, pde v loss: 1.75814
lambda obs u: 2.43356, lambda obs v: 0.41551
lambda pde u: 0.85833, lambda pde v: 0.29260
sigma_a:0:0.04790390, sigma_s:0:0.02364050, mu_a:0:0.41065896, rho_a:0:0.44821942, rho_s:0:0.04559422, kappa_a:0:0.00094819
Time taken: 26.18s

Start of epoch 58
Training observations acc over epoch: 0.0067
total loss: 24.71887, total regularisd loss: 36.03718
obs u loss: 11.51200, obs v loss: 2.11650
pde u loss: 7.73468, pde v loss: 1.72145
lambda obs u: 2.45715, lambda obs v: 0.42210
lambda pde u: 0.82123, lambda pde v: 0.29952
sigma_a:0:0.05267363, sigma_s:0:0.02420566, mu_a:0:0.44146906, rho_a:0:0.47531709, rho_s:0:0.04766666, kappa_a:0:0.00105303
Time taken: 26.12s

Start of epoch 59
Training observations acc over epoch: 0.0066
total loss: 24.51410, total regularisd loss: 35.58303
obs u loss: 11.36650, obs v loss:

Training observations acc over epoch: 0.0058
total loss: 24.59955, total regularisd loss: 31.54881
obs u loss: 9.98618, obs v loss: 1.80833
pde u loss: 6.70852, pde v loss: 1.44951
lambda obs u: 2.69049, lambda obs v: 0.45900
lambda pde u: 0.51545, lambda pde v: 0.33505
sigma_a:0:0.09035348, sigma_s:0:0.03630530, mu_a:0:0.68819712, rho_a:0:0.70967554, rho_s:0:0.05998898, kappa_a:0:0.00314015
Time taken: 26.02s

Start of epoch 77
Training observations acc over epoch: 0.0058
total loss: 25.36657, total regularisd loss: 32.06056
obs u loss: 10.11041, obs v loss: 1.80220
pde u loss: 6.87464, pde v loss: 1.45590
lambda obs u: 2.69358, lambda obs v: 0.46152
lambda pde u: 0.51123, lambda pde v: 0.33367
sigma_a:0:0.09323382, sigma_s:0:0.03703835, mu_a:0:0.69448568, rho_a:0:0.71534559, rho_s:0:0.06055034, kappa_a:0:0.00327333
Time taken: 26.01s

Start of epoch 78
Training observations acc over epoch: 0.0057
total loss: 24.95689, total regularisd loss: 31.58176
obs u loss: 9.97143, obs v loss: 1

...

Training observations acc over epoch: 0.0014
total loss: 4.44149, total regularisd loss: 5.53753
obs u loss: 2.23742, obs v loss: 0.61262
pde u loss: 1.27859, pde v loss: 0.44789
lambda obs u: 1.70847, lambda obs v: 1.00657
lambda pde u: 0.72181, lambda pde v: 0.56316
sigma_a:0:0.01867762, sigma_s:0:0.04292466, mu_a:0:0.42618420, rho_a:0:0.53701076, rho_s:0:0.08540944, kappa_a:0:0.00006802
Time taken: 25.60s

Start of epoch 419
Training observations acc over epoch: 0.0014
total loss: 4.41278, total regularisd loss: 5.51361
obs u loss: 2.19620, obs v loss: 0.59870
pde u loss: 1.25932, pde v loss: 0.44367
lambda obs u: 1.72863, lambda obs v: 0.99890
lambda pde u: 0.72681, lambda pde v: 0.54565
sigma_a:0:0.01680801, sigma_s:0:0.04352435, mu_a:0:0.42604565, rho_a:0:0.53761332, rho_s:0:0.08556969, kappa_a:0:0.00000000
Time taken: 25.60s

Start of epoch 420
Training observations acc over epoch: 0.0013
total loss: 4.26223, total regularisd loss: 5.29041
obs u loss: 2.06793, obs v loss: 0.5992

Training observations acc over epoch: 0.0012
total loss: 4.12138, total regularisd loss: 5.16491
obs u loss: 1.98266, obs v loss: 0.57252
pde u loss: 1.17830, pde v loss: 0.43395
lambda obs u: 1.76791, lambda obs v: 0.94082
lambda pde u: 0.73635, lambda pde v: 0.55492
sigma_a:0:0.01664572, sigma_s:0:0.04307890, mu_a:0:0.41472671, rho_a:0:0.52811773, rho_s:0:0.08574902, kappa_a:0:0.00000000
Time taken: 25.40s

Start of epoch 438
Training observations acc over epoch: 0.0013
total loss: 4.25172, total regularisd loss: 5.39166
obs u loss: 2.09696, obs v loss: 0.57191
pde u loss: 1.22437, pde v loss: 0.44114
lambda obs u: 1.77289, lambda obs v: 0.91609
lambda pde u: 0.75094, lambda pde v: 0.56008
sigma_a:0:0.01689877, sigma_s:0:0.04317327, mu_a:0:0.41606780, rho_a:0:0.52766394, rho_s:0:0.08397836, kappa_a:0:0.00001477
Time taken: 25.08s

Start of epoch 439
Training observations acc over epoch: 0.0013
total loss: 4.14337, total regularisd loss: 5.35056
obs u loss: 2.08409, obs v loss: 0.5761

Training observations acc over epoch: 0.0013
total loss: 4.03913, total regularisd loss: 5.32881
obs u loss: 2.11086, obs v loss: 0.56288
pde u loss: 1.15220, pde v loss: 0.41905
lambda obs u: 1.78462, lambda obs v: 0.88724
lambda pde u: 0.77704, lambda pde v: 0.55111
sigma_a:0:0.01529248, sigma_s:0:0.04223761, mu_a:0:0.40499595, rho_a:0:0.51942891, rho_s:0:0.08492835, kappa_a:0:0.00000000
Time taken: 25.39s

Start of epoch 457
Training observations acc over epoch: 0.0013
total loss: 4.02028, total regularisd loss: 5.26571
obs u loss: 2.03984, obs v loss: 0.55872
pde u loss: 1.15763, pde v loss: 0.41759
lambda obs u: 1.80413, lambda obs v: 0.87992
lambda pde u: 0.75665, lambda pde v: 0.55930
sigma_a:0:0.01548038, sigma_s:0:0.04211722, mu_a:0:0.40507176, rho_a:0:0.51863511, rho_s:0:0.08498274, kappa_a:0:0.00000000
Time taken: 25.41s

Start of epoch 458
Training observations acc over epoch: 0.0012
total loss: 3.95081, total regularisd loss: 5.10223
obs u loss: 1.95424, obs v loss: 0.5554

Training observations acc over epoch: 0.0012
total loss: 3.93760, total regularisd loss: 5.20317
obs u loss: 1.97996, obs v loss: 0.54208
pde u loss: 1.14206, pde v loss: 0.41015
lambda obs u: 1.82312, lambda obs v: 0.84393
lambda pde u: 0.76979, lambda pde v: 0.56315
sigma_a:0:0.01321332, sigma_s:0:0.04295676, mu_a:0:0.39766062, rho_a:0:0.51104172, rho_s:0:0.08377337, kappa_a:0:0.00010544
Time taken: 25.31s

Start of epoch 476
Training observations acc over epoch: 0.0013
total loss: 3.99906, total regularisd loss: 5.29292
obs u loss: 2.03377, obs v loss: 0.54483
pde u loss: 1.16143, pde v loss: 0.41064
lambda obs u: 1.80167, lambda obs v: 0.84041
lambda pde u: 0.78860, lambda pde v: 0.56932
sigma_a:0:0.01186346, sigma_s:0:0.04097848, mu_a:0:0.39685525, rho_a:0:0.51050696, rho_s:0:0.08384779, kappa_a:0:0.00022927
Time taken: 25.39s

Start of epoch 477
Training observations acc over epoch: 0.0013
total loss: 3.98136, total regularisd loss: 5.32557
obs u loss: 2.06581, obs v loss: 0.5475

Training observations acc over epoch: 0.0012
total loss: 3.79373, total regularisd loss: 5.08508
obs u loss: 1.97714, obs v loss: 0.53835
pde u loss: 1.10479, pde v loss: 0.39329
lambda obs u: 1.79130, lambda obs v: 0.81524
lambda pde u: 0.80916, lambda pde v: 0.58430
sigma_a:0:0.01057321, sigma_s:0:0.04045829, mu_a:0:0.38430736, rho_a:0:0.50346525, rho_s:0:0.08533691, kappa_a:0:0.00000000
Time taken: 25.78s

Start of epoch 495
Training observations acc over epoch: 0.0013
total loss: 3.82759, total regularisd loss: 5.18567
obs u loss: 2.02147, obs v loss: 0.54604
pde u loss: 1.10175, pde v loss: 0.39014
lambda obs u: 1.80422, lambda obs v: 0.81420
lambda pde u: 0.81258, lambda pde v: 0.56900
sigma_a:0:0.01075665, sigma_s:0:0.04189940, mu_a:0:0.38455860, rho_a:0:0.50217767, rho_s:0:0.08434196, kappa_a:0:0.00000000
Time taken: 25.52s

Start of epoch 496
Training observations acc over epoch: 0.0012
total loss: 3.68501, total regularisd loss: 4.91790
obs u loss: 1.88420, obs v loss: 0.5137

In [6]:
32*512

16384